In [2]:
from pymongo import MongoClient

In [3]:
client = MongoClient('mongodb://localhost:27017')

In [4]:
bd = client.biblioteca

In [5]:
from timeit import Timer

![Ejemplo estructura](http://docs.mongodb.org/manual/_images/data-model-tree.png)

##Model Tree Structures with Parent References (Parent Links)
Almacena cada nodo del árbol en un documento; adicionalmente a la información del nodo del árbol el documento almacena el 'id' of the nodo padre.

| Pros        | Cons           
| ------------- |:-------------:|
| Consulta para recuperar el padre de un nodo más rápida y sencilla|   |
| Provee una solución simple para el almacenamiento | Se requiere múltimples consultas para recuperar subárboles|
| Se puede crear un índice sobre el campo 'parent' para permitir búsquedas rápidas |       |
| Se puede consultar por el campo 'parent' para encontrar inmediatamente los nodos hijos |       |

In [7]:
def test_parent_references():
    bd.categorias.drop()
    bd.categorias.insert( { '_id': "MongoDB", 'parent': "Databases" } ) 
    bd.categorias.insert( { '_id': "dbm", 'parent': "Databases" } ) 
    bd.categorias.insert( { '_id': "Databases", 'parent': "Programming" } ) 
    bd.categorias.insert( { '_id': "Languages", 'parent': "Programming" } ) 
    bd.categorias.insert( { '_id': "Programming", 'parent': "Books" } ) 
    bd.categorias.insert( { '_id': "Books", 'parent': None } )

In [8]:
t = Timer(lambda: test_parent_references())
time = t.timeit(number=100)
print('%.2f ' % time)

0.31 


In [25]:
bd.categorias.find_one({'_id': 'MongoDB'}).get('parent')

'Databases'

In [29]:
bd.categorias.create_index('parent')

'parent_1'

In [31]:
cursor = bd.categorias.find({'parent' : 'Databases'})
for c in cursor:
    print(c)

{'_id': 'MongoDB', 'parent': 'Databases'}
{'_id': 'dbm', 'parent': 'Databases'}


##Model Tree Structures with Child References

El patrón "Child References" almacena cada nodo del árbol en un documento; adicionalmente a la información del nodo de árbol, el documento almacena en un arreglo los id(s) de los nodos hijos.

In [32]:
def test_parent_references():
    bd.categorias.drop()
    bd.categorias.insert( { "_id": "MongoDB", "children": [] } ) 
    bd.categorias.insert( { "_id": "dbm", "children": [] } ) 
    bd.categorias.insert( { "_id": "Databases", "children": ["MongoDB", "dbm"] } ) 
    bd.categorias.insert( { "_id": "Languages", "'children": [] } ) 
    bd.categorias.insert( { "_id": "Programming", "children": ["Databases", "Languages"] } ) 
    bd.categorias.insert( { "_id": "Books", "children": ["Programming"] } )